Here’s a QMD blog post about langgraph, including code examples that should run in your environment:

# Building Complex LLM Workflows with LangGraph

For **language models (LLMs)**, managing complex workflows, conditional logic, and stateful execution is challenging. That's where **LangGraph** comes in. It enables you to build sophisticated, **stateful**, and **modular** applications using a **graph-based** approach, making it ideal for multi-agent systems, conditional flows, and more.

## What is LangGraph?

`LangGraph` is a Python framework designed to build **stateful, multi-agent applications**. It structures your LLM workflow as a **directed acyclic graph (DAG)** where each node represents a function, model call, or tool. These nodes are connected with edges that define how data flows from one node to another, allowing you to create complex, conditional workflows.

## Key Features

- **Graph-based execution**: Unlike a linear chain of tasks (like in LangChain), LangGraph allows for explicit, flexible, graph-based routing. You can branch logic or parallelize tasks.
- **State management**: LangGraph uses a centralized, mutable state that flows through the graph, enabling easy tracking of evolving information.
- **Concurrency**: The framework supports parallel execution of branches, making it great for handling multiple agents or parallel tasks.

## Core Components

LangGraph is centered around a few key objects:

- **`StateGraph`**: This is the core class used to build and compile the graph.
- **`END`**: A special constant to signal graph termination.
- **`State`**: The state that flows through the graph. It's typically a dictionary or a Pydantic model.
- **`ToolNode`, `FunctionNode`, `LLMNode`**: These represent various types of nodes, which can be LLM calls, tools, or functions.

## Creating a Graph

Here’s a simple example to get started:


In [ ]:
from langgraph import StateGraph, State, FunctionNode, END

# Define a state type
class MyState(State):
    count: int

# Create a graph
sg = StateGraph(MyState)

# Add nodes (functions in this case)
def increment(state: MyState) -> MyState:
    state.count += 1
    return state

# Add the increment node
sg.add_node("increment", increment)

# Set the entry point
sg.set_entry_point("increment")

# Compile the graph into an executable
graph = sg.compile()

## Run the graph


In [ ]:
result = graph.invoke(MyState(count=0))
print(result.count)  # Output should be 1

In this simple example:
	•	We define a state (MyState) that holds a counter.
	•	We create a node (increment) that increments this counter.
	•	We compile and run the graph, outputting the incremented count.

## Advanced Features

Conditional Logic and Branching

You can add conditional logic to decide the next node based on the state or results from previous nodes. For example:


In [ ]:
def check_even(state: MyState) -> State:
    if state.count % 2 == 0:
        state.count += 2  # Even, increment by 2
    else:
        state.count -= 1  # Odd, decrement by 1
    return state

# Modify the graph with conditional logic
sg.add_node("check_even", check_even)
sg.add_edge("increment", "check_even")
sg.add_edge("check_even", END)

graph = sg.compile()
result = graph.invoke(MyState(count=0))  # Starts with 0, will increment by 2 to 2
print(result.count)  # Output: 2

Here, after incrementing the count, the state is checked for evenness. If even, it increments by 2, otherwise decrements by 1.

## When to Use LangGraph

LangGraph shines in applications that require:
	•	Multi-agent systems: Where different agents interact with each other and make decisions based on shared state.
	•	Stateful workflows: When you need to maintain and pass state across multiple steps (e.g., conversational agents).
	•	Conditional and dynamic logic: For workflows where the path changes depending on previous outputs.
	•	Complex LLM pipelines: Combining multiple LLMs, tools, and decision-making logic into a single process.

## Comparison with LangChain

Although LangChain is also a popular framework for building LLM applications, it works on a more linear or recursive basis, whereas LangGraph uses a graph-based approach that offers greater flexibility and modularity. Here’s a quick comparison:

| Feature               | LangGraph                               | LangChain                                 |
|-----------------------|-----------------------------------------|-------------------------------------------|
| Core abstraction      | Directed graph of nodes                 | Linear or recursive chain of steps        |
| Execution flow        | Explicit, flexible, graph-based routing | Mostly linear or tree-like flow           |
| State management      | Centralized mutable state passed through nodes | Local inputs/outputs passed between chains |
| Concurrency           | Supports parallel branches              | Limited concurrency support               |
| Use case focus        | Multi-agent systems, conditional flows  | LLM wrappers, tools, agents, retrievers   |

## Conclusion

LangGraph brings a powerful way to structure complex workflows in LLM-based systems. It excels in scenarios with branching logic, multi-agent interactions, and long-lived stateful applications. If you’re building systems that require more than just simple chains of LLM calls, LangGraph can provide the flexibility and control you need.
